In [1]:
!pip install xgboost

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

In [3]:
data = pd.read_csv(r"C:\Users\revan\Downloads\Predict Student Performance.csv")

In [7]:
data.head()

,Socioeconomic Score,Study Hours,Sleep Hours,Attendance (%),Grades
0,0.95822,3.4,8.2,53.0,47.0
1,0.85566,3.2,5.9,55.0,35.0
2,0.68025,3.2,9.3,41.0,32.0
3,0.25936,3.2,8.2,47.0,34.0
4,0.60447,3.8,10.0,75.0,33.0


In [11]:
data.tail()

,Socioeconomic Score,Study Hours,Sleep Hours,Attendance (%),Grades
1383,0.44549,5.5,8.0,51.0,41.0
1384,0.52466,4.9,6.5,63.0,37.0
1385,0.88197,3.9,6.2,54.0,36.0
1386,0.47336,3.5,7.3,61.0,34.0
1387,0.58119,3.7,9.7,79.0,35.0


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1388 entries, 0 to 1387
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Socioeconomic Score  1388 non-null   float64
 1   Study Hours          1388 non-null   float64
 2   Sleep Hours          1388 non-null   float64
 3   Attendance (%)       1388 non-null   float64
 4   Grades               1388 non-null   float64
dtypes: float64(5)
memory usage: 54.3 KB


In [15]:
data.describe()

,Socioeconomic Score,Study Hours,Sleep Hours,Attendance (%),Grades
count,1388.000000,1388.000000,1388.000000,1388.000000,1388.000000
mean,0.552274,4.560807,8.047262,58.536023,40.691643
std,0.261272,1.897581,1.370700,11.675287,9.467358
min,0.101280,0.800000,4.800000,40.000000,32.000000
25%,0.322118,3.475000,7.000000,49.000000,34.000000
50%,0.545945,3.900000,8.400000,57.000000,35.000000
75%,0.789610,5.900000,9.100000,66.000000,47.000000
max,0.999820,10.000000,10.000000,100.000000,91.000000


In [17]:
data.shape

(1388, 5)

In [19]:
data.isnull()

,Socioeconomic Score,Study Hours,Sleep Hours,Attendance (%),Grades
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
1383,False,False,False,False,False
1384,False,False,False,False,False
1385,False,False,False,False,False
1386,False,False,False,False,False


In [21]:
data.isna().sum()

Socioeconomic Score    0
Study Hours            0
Sleep Hours            0
Attendance (%)         0
Grades                 0
dtype: int64

In [14]:
# Bin "Grades" into categories (e.g., Pass/Fail based on a threshold)
threshold = 50  # Example threshold for passing
data["Grade Category"] = data["Grades"].apply(lambda x: "Pass" if x >= threshold else "Fail")

In [16]:
# Encode the target variable
label_encoder = LabelEncoder()
data["Grade Category"] = label_encoder.fit_transform(data["Grade Category"])

In [18]:
# Define features and target
features = data[["Socioeconomic Score", "Study Hours", "Sleep Hours", "Attendance (%)"]]
target = data["Grade Category"]

In [20]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

In [22]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [24]:
# Create and train the XGBoost classifier
xgb_classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_classifier.fit(X_train_scaled, y_train)

C:\Users\revan\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:37:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [26]:
# Predict on the test set
y_pred = xgb_classifier.predict(X_test_scaled)

In [28]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [30]:
# Print results
print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:\n")
print(report)

Accuracy: 96.88%

Classification Report:

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       344
           1       0.91      0.92      0.91        73

    accuracy                           0.97       417
   macro avg       0.94      0.95      0.95       417
weighted avg       0.97      0.97      0.97       417

